In [2]:
# Start writing code here...
import pandas as pd

In [32]:
df = pd.read_csv("matrice.csv")
df.head()

,SIRET,Code_postal,Numero_inspection,Date_inspection,Agrement,synthese_eval_encode,month,month_sin,month_cos,day,day_sin,day_cos,activite_category,departement
0,30247464800789,92240,15914760,2021-03-03 00:00:00+00:00,0,1.0,3,1.000000,6.123234e-17,3,0.051620,0.998667,1,92
1,30112032500024,69500,16252422,2021-11-26 00:00:00+00:00,0,0.0,11,-0.500000,8.660254e-01,11,0.188227,0.982126,58,69
2,30247464801894,92800,15903092,2021-02-22 00:00:00+00:00,0,0.0,2,0.866025,5.000000e-01,2,0.034422,0.999407,1,92
3,30890741900011,78460,15901554,2021-02-19 00:00:00+00:00,0,1.0,2,0.866025,5.000000e-01,2,0.034422,0.999407,72,78
4,30668909200022,94290,15968279,2021-04-14 00:00:00+00:00,0,1.0,4,0.866025,-5.000000e-01,4,0.068802,0.997630,1,94


In [31]:
for i in df:
    try:
        df[i] = df[i].astype(int)
    except:
        print(i)

Numero_inspection
Date_inspection


In [34]:
data = df.copy()
data = data.drop(['SIRET', 'Numero_inspection', 'Date_inspection'], axis=1)
data.head()

,Code_postal,Agrement,synthese_eval_encode,month,month_sin,month_cos,day,day_sin,day_cos,activite_category,departement
0,92240,0,1.0,3,1.000000,6.123234e-17,3,0.051620,0.998667,1,92
1,69500,0,0.0,11,-0.500000,8.660254e-01,11,0.188227,0.982126,58,69
2,92800,0,0.0,2,0.866025,5.000000e-01,2,0.034422,0.999407,1,92
3,78460,0,1.0,2,0.866025,5.000000e-01,2,0.034422,0.999407,72,78
4,94290,0,1.0,4,0.866025,-5.000000e-01,4,0.068802,0.997630,1,94


In [35]:
from sklearn.model_selection import train_test_split


# Define our Independent(predictor) and Dependent(response) variables.
X = data.drop(['synthese_eval_encode'],axis=1)
y = data['synthese_eval_encode']

# Split our dataset into train and test subsets.
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20,random_state=42)

In [39]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()
X_train_fit = reg.fit(X_train, y_train)
reg.score(X_train, y_train)

0.03917262719835135

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6a1379aa-ea9c-402c-95ca-194460431a68' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>